In [ ]:
'''Python Introductions provided by FireWorks'''

In [ ]:
'''How FireWorks performs calculations'''
from fireworks import Firework, FWorker, LaunchPad
from fireworks.core.rocket_launcher import launch_rocket
from fw_tutorials.firetask.addition_task import AdditionTask

# set up the LaunchPad and reset it
launchpad = LaunchPad()
# launchpad.reset('', require_password=False)

# create the Firework consisting of a custom "Addition" task
firework = Firework(AdditionTask(), spec={"input_array": [1, 2]})

# store workflow and launch it locally
launchpad.add_wf(firework)
launch_rocket(launchpad, FWorker())

In [ ]:
'''How to define a FireWork's Workflow'''
from fireworks import Firework, Workflow, FWorker, LaunchPad, ScriptTask
from fireworks.core.rocket_launcher import rapidfire
launchpad = LaunchPad()
# launchpad.reset('', require_password=False)

# define four individual FireWorks used in the Workflow
task1 = ScriptTask.from_str('echo "Ingrid is the CEO."')
task2 = ScriptTask.from_str('echo "Jill is a manager."')
task3 = ScriptTask.from_str('echo "Jack is a manager."')
task4 = ScriptTask.from_str('echo "Kip is an intern."')

fw1 = Firework(task1)
fw2 = Firework(task2)
fw3 = Firework(task3)
fw4 = Firework(task4)

# assemble Workflow from FireWorks and their connections by id
workflow = Workflow([fw1, fw2, fw3, fw4], {fw1: [fw2, fw3], fw2: [fw4], fw3: [fw4]})

# store workflow and launch it locally
launchpad.add_wf(workflow)
rapidfire(launchpad, FWorker())

In [ ]:
'''Defining a dynamic Workflow with PyTasks'''
from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction
from fireworks.core.rocket_launcher import rapidfire
from fw_tutorials.dynamic_wf.fibadd_task import FibonacciAdderTask

# FireWorks needs to be able to find functions inside your python path, find the module name for this file
mod_name = __name__

# Define a new python function that will be used in a PyTask
def fibonacci_adder(num1, num2, stop_point=0):
    if num1 + num2 < stop_point:
        new_fw = Firework(PyTask({'func': mod_name+'.fibonacci_adder', 'args': [num2, num1 + num2, stop_point]}))
        print(f"The current number is {num1+num2}")
        # Return a FWAction that will append a new Firework at the end of the Workflow
        return FWAction(additions=new_fw)
    else:
        print(f"The limit was reached, the next number is {num1+num2}")

launchpad = LaunchPad(strm_lvl=loglvl, port=27018)
firework = Firework(PyTask({"func":mod_name+'.fibonacci_adder', "args":[0, 1, 1000]}))
launchpad.add_wf(firework)
# rapidfire will keep running Fireworks until there are no ready jobs in the LaunchPad
rapidfire(launchpad, FWorker(), strm_lvl=loglvl)